In [1]:
from pymongo import MongoClient
import pandas as pd
import time

# Connect to db

In [2]:
client = MongoClient('localhost', 27017)
db = client.gfif

In [3]:
db.eval("db.stats()")

{'avgObjSize': 945.9603330269452,
 'collections': 4,
 'dataFileVersion': {'major': 4, 'minor': 5},
 'dataSize': 92469372580.0,
 'db': 'gfif',
 'fileSize': 128720044032.0,
 'indexSize': 28523660256.0,
 'indexes': 5,
 'nsSizeMB': 16,
 'numExtents': 66,
 'objects': 97751850,
 'ok': 1.0,
 'storageSize': 94625086832.0}

In [148]:
%%writefile drive.cfg
[FILES]
DOIS.xlsx               = 1bikNT7Gmp4G7dfeMuGsF-az7D8lskK0O
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x
datos1.csv              = 11CyLRZZwVbgw6YAC-igRJ3mrkIwk0aaiXnd-EOofYTI
oadoi_udea.json         = 1s508qokcIkz-i1Y4Ukbdht-jWuSVngON

Overwriting drive.cfg


In [149]:
import wosplus as wp
files=wp.wosplus('drive.cfg')

In [50]:
udea=files.read_drive_excel('oaudea.xlsx')

In [153]:
wp.download_file_from_google_drive( files.drive_file.get('oadoi_udea.json') )

In [151]:
files.read_drive_excel??

In [68]:
results=db.papers.find({"doi":{'$in':udea.DI.drop_duplicates().values.tolist()}})

In [70]:
df = pd.DataFrame(list(results)).drop('_id',axis='columns')

In [136]:
df.to_json('oadoi_udea.json')

In [140]:
df=pd.read_json('oadoi_udea.json')

In [141]:
dfin=df[df.best_oa_location.isnull()].reset_index(drop=True)

In [142]:
dfinn=df[~df.best_oa_location.isnull()].reset_index(drop=True)

In [143]:
dfinn.best_oa_location[0]

{'evidence': 'open (via free pdf)',
 'host_type': 'publisher',
 'is_best': True,
 'license': None,
 'pmh_id': None,
 'updated': '2017-09-02T17:22:49.666964',
 'url': 'http://archinte.jamanetwork.com/data/journals/intemed/5513/ioi50162.pdf',
 'url_for_landing_page': 'http://doi.org/10.1001/archinte.166.1.95',
 'url_for_pdf': 'http://archinte.jamanetwork.com/data/journals/intemed/5513/ioi50162.pdf',
 'version': 'publishedVersion'}

In [144]:
dfinn['url_for_pdf']=dfinn.best_oa_location.apply(lambda x: x.get('url_for_pdf'))

In [145]:
dfin['url_for_pdf']=None

In [146]:
dff=dfinn.append(dfin).reset_index(drop=True)

In [147]:
dff

,best_oa_location,data_standard,doi,doi_url,genre,is_oa,journal_is_in_doaj,journal_is_oa,journal_issns,journal_name,oa_locations,published_date,publisher,title,updated,year,z_authors,url_for_pdf
0,"{'pmh_id': None, 'updated': '2017-09-02T17:22:...",2,10.1001/archinte.166.1.95,https://doi.org/10.1001/archinte.166.1.95,journal-article,True,False,False,0003-9926,Archives of Internal Medicine,"[{'pmh_id': None, 'updated': '2017-09-02T17:22...",2006-01-09,American Medical Association (AMA),Etiologic Diagnosis of Chronic Osteomyelitis,2018-06-15T03:19:43.547479,2006.0,"[{'given': 'Andres F.', 'family': 'Zuluaga'}]",http://archinte.jamanetwork.com/data/journals/...
1,"{'pmh_id': None, 'updated': '2017-09-27T16:19:...",2,10.1001/archneur.55.6.857,https://doi.org/10.1001/archneur.55.6.857,journal-article,True,False,False,0003-9942,Archives of Neurology,"[{'pmh_id': None, 'updated': '2017-09-27T16:19...",1998-06-01,American Medical Association (AMA),Patients With Epilepsy Who Die Suddenly Have C...,2018-06-19T02:56:53.795724,1998.0,"[{'given': 'Benjamin H.', 'family': 'Natelson'...",http://archneur.jamanetwork.com/data/journals/...
2,"{'pmh_id': None, 'updated': '2017-08-31T16:39:...",2,10.1001/jamaneurol.2015.4851,https://doi.org/10.1001/jamaneurol.2015.4851,journal-article,True,False,False,2168-6149,JAMA Neurology,"[{'pmh_id': None, 'updated': '2017-08-31T16:39...",2016-04-01,American Medical Association (AMA),Cognitive Decline in a Colombian Kindred With ...,2018-06-15T19:01:39.356347,2016.0,"[{'given': 'Daniel C.', 'family': 'Aguirre-Ace...",http://archneur.jamanetwork.com/data/journals/...
3,"{'pmh_id': None, 'updated': '2018-06-21T01:54:...",2,10.1016/j.actatropica.2017.10.008,https://doi.org/10.1016/j.actatropica.2017.10.008,journal-article,True,False,False,0001-706X,Acta Tropica,"[{'pmh_id': None, 'updated': '2018-06-21T01:54...",2018-01-01,Elsevier BV,The Akt- like kinase of Leishmania panamensis ...,2018-06-18T11:13:31.462344,2018.0,"[{'given': 'Didier', 'family': 'Tirado-Duarte'...",None
4,{'pmh_id': 'oai:scielo:S0210-48062011000200013...,2,10.1016/j.acuro.2010.10.004,https://doi.org/10.1016/j.acuro.2010.10.004,journal-article,True,False,False,0210-4806,Actas Urológicas Españolas,[{'pmh_id': 'oai:scielo:S0210-4806201100020001...,2011-02-01,Elsevier BV,"Tenofovir, agente antiviral con escasa activid...",2018-06-17T04:52:30.698996,2011.0,"[{'given': 'L.', 'family': 'Ospina'}, {'given'...",http://scielo.isciii.es/pdf/aue/v35n2/carta1.pdf
5,"{'pmh_id': None, 'updated': '2018-06-21T05:17:...",2,10.1016/j.ajhg.2010.04.016,https://doi.org/10.1016/j.ajhg.2010.04.016,journal-article,True,False,False,0002-9297,The American Journal of Human Genetics,"[{'pmh_id': None, 'updated': '2018-06-21T05:17...",2010-06-01,Elsevier BV,Loss-of-Function Mutations in HPSE2 Cause the ...,2018-06-15T23:22:18.708304,2010.0,"[{'given': 'Junfeng', 'family': 'Pang'}, {'giv...",None
6,"{'pmh_id': None, 'updated': '2018-06-18T19:42:...",2,10.1016/j.ajhg.2012.04.015,https://doi.org/10.1016/j.ajhg.2012.04.015,journal-article,True,False,False,0002-9297,The American Journal of Human Genetics,"[{'pmh_id': None, 'updated': '2018-06-18T19:42...",2012-06-01,Elsevier BV,Deleterious Mutations in LRBA Are Associated w...,2018-06-18T19:42:04.632633,2012.0,"[{'given': 'Gabriela', 'family': 'Lopez-Herrer...",None
7,"{'pmh_id': None, 'updated': '2018-06-19T08:43:...",2,10.1016/j.ajhg.2016.02.014,https://doi.org/10.1016/j.ajhg.2016.02.014,journal-article,True,False,False,0002-9297,The American Journal of Human Genetics,"[{'pmh_id': None, 'updated': '2018-06-19T08:43...",2016-04-01,Elsevier BV,A Genome-wide Association Study of Nonsyndromi...,2018-06-19T08:43:07.608541,2016.0,"[{'given': 'Elizabeth J.', 'family': 'Leslie'}...",None
8,{'pmh_id': 'oai:pubmedcentral.nih.gov:5673667'...,2,10.1016/j.ajhg.2017.09.021,https://doi.org/10.1016/j.ajhg.2017.09.021,journal-article,True,False,False,0002-9297,The American Journal of Human Genetics,[{'pmh_id': 'oai:pubmedcentral.nih.gov:567